<a href="https://colab.research.google.com/github/abdulrahmanalaa123/Graduation_Project/blob/Honda/grad_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

imports


In [2]:
!pip3 install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 256 kB 5.0 MB/s 


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from re import L
import json
from operator import itemgetter
from google.colab import files
import pickle5 as pickle
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from sklearn.model_selection import train_test_split





In [5]:
uploaded = files.upload()


Saving hr(tuples).pkl to hr(tuples).pkl


In [6]:
with open("hr(tuples).pkl", "rb") as fh:
  data = pickle.load(fh)

In [105]:
#data = pd.read_pickle("hr(tuples).pkl")

In [12]:
print(data)

      Participant           Stage       0:0.0  0:0.099791  0:0.199582  \
0              22        STIMULUS   (87, 602)   (87, 602)   (87, 602)   
1              22  QUESTIONNAIRES   (85, 921)   (85, 921)   (85, 921)   
2              22         WASHOUT   (80, 749)   (80, 792)   (80, 792)   
3              22        STIMULUS   (81, 719)   (81, 719)   (81, 719)   
4              22  QUESTIONNAIRES   (83, 702)   (82, 702)   (83, 702)   
...           ...             ...         ...         ...         ...   
1339           64        STIMULUS   (78, 516)   (78, 516)   (78, 516)   
1340           64  QUESTIONNAIRES   (77, 799)   (77, 799)   (77, 816)   
1341           64         WASHOUT    (0, 987)    (0, 987)    (0, 987)   
1342           64        STIMULUS   (69, 829)   (68, 829)   (69, 829)   
1343           64  QUESTIONNAIRES  (66, 1245)  (65, 1245)  (65, 1245)   

     0:0.299373  0:0.399164 0:0.49895500000000004  0:0.598746  \
0     (87, 602)   (87, 602)             (87, 602)   (87, 6

In [7]:
def getter(x):
    if isinstance(x,tuple):
        return itemgetter(0)(x)
    else:
        return x
        
#data = pd.read_pickle("hr(tuples).pkl")
#mask = data["Participant"] ==  22 
mask2 = data["Stage"] == "STIMULUS"
mask3 = data["Target"]== "BASELINE"
#print(data.loc[ mask2,[*data.columns[2:500],*data.columns[-15:]]].applymap(lambda x:getter(x)))
#print(data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x)))#xtrain->particpants hr values
print(data.loc[mask2,"Target"])#ytrain-> target emo


0        BASELINE
3         NEUTRAL
6             AWE
9         DISGUST
12       SURPRISE
          ...    
1330         FEAR
1333    AMUSEMENT
1336      SADNESS
1339      NEUTRAL
1342          AWE
Name: Target, Length: 462, dtype: object


In [17]:
xtrain=data.loc[mask2,[*data.columns[2:6292]]].applymap(lambda x:getter(x))
ytrain=data.loc[mask2,"Target"]

In [23]:
x_train=[]
y_train=[]
label_encoder = LabelEncoder()
x_train,y_train=np.array(xtrain),np.array(ytrain)
y_train = label_encoder.fit_transform(y_train)
label_encoder_name_mapping = dict(zip(label_encoder.classes_,
                                         label_encoder.transform(label_encoder.classes_)))
x_train,  x_test, y_train, y_test = train_test_split(x_train, y_train, train_size = 0.7, random_state = 123)
x_train,x_test=np.reshape(x_train,(x_train.shape[0],x_train.shape[1],1)),np.reshape(x_test,(x_test.shape[0],x_test.shape[1],1))
#print(label_encoder_name_mapping)

In [24]:
#initalizing the RNN
regressor =Sequential()

In [25]:
#adding the first lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True,input_shape=(x_train.shape[1],1)))
regressor.add(Dropout(0.2))

In [26]:
#adding the 2nd lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [27]:
#adding the 3rd lstm layer and some dropout regularisation
regressor.add(LSTM(units=50,return_sequences=True))
regressor.add(Dropout(0.2))

In [28]:
#adding the 4th lstm layer and some dropout regularisation
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.2))

In [29]:
#adding the output layer
regressor.add (Dense(units=11,activation='softmax'))

In [30]:
#compiling the RNN
regressor.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics='accuracy')

In [31]:
print(regressor.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 6290, 50)          10400     
                                                                 
 dropout_1 (Dropout)         (None, 6290, 50)          0         
                                                                 
 lstm_2 (LSTM)               (None, 6290, 50)          20200     
                                                                 
 dropout_2 (Dropout)         (None, 6290, 50)          0         
                                                                 
 lstm_3 (LSTM)               (None, 6290, 50)          20200     
                                                                 
 dropout_3 (Dropout)         (None, 6290, 50)          0         
                                                                 
 lstm_4 (LSTM)               (None, 50)               

In [32]:
#Fitting the RNN to the training set
regressor.fit(x_train,y_train,epochs=100,batch_size=50,callbacks=[
    EarlyStopping(monitor='val_loss',
    patience=5,
    restore_best_weights=True)
])

Epoch 1/100
7/7 [==============================] - ETA: 0s - loss: 2.3990 - accuracy: 0.0526 

7/7 [==============================] - 180s 24s/step - loss: 2.3990 - accuracy: 0.0526
Epoch 2/100
7/7 [==============================] - ETA: 0s - loss: 2.3957 - accuracy: 0.0991 

7/7 [==============================] - 163s 23s/step - loss: 2.3957 - accuracy: 0.0991
Epoch 3/100
7/7 [==============================] - ETA: 0s - loss: 2.3936 - accuracy: 0.0929 

7/7 [==============================] - 163s 23s/step - loss: 2.3936 - accuracy: 0.0929
Epoch 4/100
7/7 [==============================] - ETA: 0s - loss: 2.3928 - accuracy: 0.1022 

7/7 [==============================] - 168s 24s/step - loss: 2.3928 - accuracy: 0.1022
Epoch 5/100
7/7 [==============================] - ETA: 0s - loss: 2.3904 - accuracy: 0.0836 

...

7/7 [==============================] - 166s 24s/step - loss: 2.3896 - accuracy: 0.1269
Epoch 12/100
7/7 [==============================] - ETA: 0s - loss: 2.3849 - accuracy: 0.1022 

7/7 [==============================] - 166s 23s/step - loss: 2.3849 - accuracy: 0.1022
Epoch 13/100
7/7 [==============================] - ETA: 0s - loss: 2.3867 - accuracy: 0.0960 

7/7 [==============================] - 173s 25s/step - loss: 2.3867 - accuracy: 0.0960
Epoch 14/100
7/7 [==============================] - ETA: 0s - loss: 2.3859 - accuracy: 0.1115 

7/7 [==============================] - 168s 24s/step - loss: 2.3859 - accuracy: 0.1115
Epoch 15/100
4/7 [================>.............] - ETA: 1:13 - loss: 2.3805 - accuracy: 0.1000

KeyboardInterrupt: ignored

In [ ]:
model_acc=regressor.evaluate(x_test,y_test,verbose=0)[1]
print("test acc: {:.3f}%".format(model_acc * 100)